In [7]:
import crawler as cw
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
DRIVER = cw.initiate_crawler()

In [12]:
def get_soup_news(driver, query):
    news_search_url = 'https://www.google.com/search?q="'+str(query) +'"&tbm=nws'

    driver.get(news_search_url)
    driver.set_page_load_timeout(10)

    soup = bs(driver.page_source, 'lxml')
    return soup

In [15]:
def extract_info(soup):
    news = soup.find_all('a', attrs={'class': 'WlydOe'}, href=True)
    title_class = ["mCBkyc y355M JQe2Ld nDgy9d", "mCBkyc y355M nDgy9d"]
    desc_class="GI74Re nDgy9d" #desc
    
    df = pd.DataFrame()
    links, titles, descs = [], [], []
    for n in news:
        links.append(n['href'])
        
        title = n.find('div', attrs={'class': title_class[0]})
        if(title):
            title = title.get_text()
        else:
            title = n.find('div', attrs={'class': title_class[1]}).get_text()
        
        titles.append(title)
        
        des = n.find('div', attrs={'class': desc_class})
        if(des):
            des = des.get_text()
        else:
            des = "none"
            
        descs.append(des)
        
    
    df['title'] = titles
    df['desc'] = descs
    df['link'] = links
    
    return df

## Testing

In [14]:
news_soup = get_soup_news(DRIVER, "motogp mandalika")

In [16]:
extract_info(news_soup)

,title,desc,link
0,Jadwal Lengkap MotoGP Mandalika 2022 di Sirkui...,Berikut jadwal MotoGP Mandalika 2022 pada 1820...,https://sports.okezone.com/read/2022/03/17/38/...
1,Hasil FP2 MotoGP Mandalika: Quartararo Tercepa...,KOMPAS.com - Sesi latihan bebas alias free pra...,https://www.kompas.com/motogp/read/2022/03/18/...
2,Hasil FP2 MotoGP Mandalika: Quartararo Tercepa...,TRIBUNNEWS.COM - Fabio Quartararo menjadi yang...,https://www.tribunnews.com/sport/2022/03/18/ha...
3,Joan Mir: Sirkuit Mandalika Sudah Bersih,"Joan Mir puas dengan Sirkuit Mandalika. Kini, ...",https://sport.detik.com/moto-gp/d-5989907/joan...
4,Begini Fasilitas Nonton MotoGP Mandalika Harga...,Berbagai kelas tiket MotoGP Mandalika 2022 dij...,https://www.cnnindonesia.com/olahraga/20220318...
5,Pembalap asal Spanyol Dengar Sholawat Tarhim d...,Pembalap Moto2 asal Spanyol Marcos Ramirez tak...,https://sports.okezone.com/read/2022/03/17/38/...
6,2 Link Live Streaming MotoGP Mandalika Indones...,Simak jadwal MotoGP Mandalika Indonesia 2022 l...,https://beritadiy.pikiran-rakyat.com/citizen/p...
7,Penyiraman Lintasan Sirkuit Mandalika Sempat D...,Penyiraman lintasan Sirkuit Mandalika dilakuka...,https://www.kompas.com/motogp/read/2022/03/17/...
8,"Jatuh di Sirkuit Mandalika, Marc Marquez Baik-...",Marc Marquez terjatuh kala sesi Free Practice ...,https://sport.detik.com/moto-gp/d-5989771/jatu...
9,Jumlah Penumpang Pesawat dari Makassar ke Lomb...,none,https://makassar.sindonews.com/read/716669/710...
